In [17]:
import random
import heapq
import itertools
import math
import copy

def mazegen(height, width, obstacle_weight=0.2):
    """
    Generates a grid of '.' and '#'.
    Obstacle weight is the probability that each space will be # (0.2 by default).
    Sets top left and bottom right spaces to be clear.
    Does not guarantee a clear path between them!
    """
    maze = [[random.choices(['.', '#'], weights=[1 - obstacle_weight, obstacle_weight])[0]
             for _ in range(width)] for _ in range(height)]
    maze[0][0] = '.'
    maze[-1][-1] = '.'
    return maze

def print_maze(maze):
    print('\n'.join([''.join(i) for i in maze]))

def valid_space(maze, space):
    return 0 <= space[0] < len(maze) \
           and 0 <= space[1] < len(maze[0]) \
           and maze[space[0]][space[1]] == '.'

class PriorityQueue:
    """Adapted from the implementation notes on https://docs.python.org/3.8/library/heapq.html"""
    def __init__(self):
        self.heap = []
        self.entry_finder = {}
        self.REMOVED = '<removed-task>'  # placeholder for a removed task
        self.counter = itertools.count()

    def push(self, task, priority=0):
        'Add a new task or update the priority of an existing task'
        if task in self.entry_finder:
            self.remove(task)
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heapq.heappush(self.heap, entry)

    def remove(self, task):
        'Mark an existing task as REMOVED. Raise KeyError if not found.'
        entry = self.entry_finder.pop(task)
        entry[-1] = self.REMOVED

    def pop(self):
        'Remove and return the lowest priority task. Raise KeyError if empty.'
        while self.heap:
            priority, count, task = heapq.heappop(self.heap)
            if task is not self.REMOVED:
                del self.entry_finder[task]
                return task
        raise KeyError('pop from an empty priority queue')

    def contains(self, state):
        return state in self.entry_finder

    def length(self):
        return len(self.heap)

    def get_priority(self, task):
        """Returns math.inf if the task is not in the queue"""
        if task in self.entry_finder:
            return self.entry_finder[task][0]
        else:
            return math.inf

class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.path_cost = parent.path_cost + 1 if parent else 0

    def __eq__(self, other):
        return self.state == other.state

    def __lt__(self, other):
        return self.state < other.state

    def __hash__(self):
        return hash(self.state)

def greedy_search(maze, start=(0, 0), goal=None):
    if goal is None:
        goal = (len(maze) - 1, len(maze[0]) - 1)
    
    heuristic = lambda node: abs(goal[0] - node.state[0]) + abs(goal[1] - node.state[1])
    frontier = PriorityQueue()
    frontier.push(Node(start), priority=heuristic(Node(start)))
    explored = set()

    number_explored = 0
    while frontier.length() > 0:
        current_node = frontier.pop()
        current_state = current_node.state

        number_explored += 1
        if current_state == goal:
            return current_node, number_explored

        explored.add(current_state)
        for move in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            neighbor = (current_state[0] + move[0], current_state[1] + move[1])
            if valid_space(maze, neighbor) and neighbor not in explored:
                neighbor_node = Node(neighbor, parent=current_node)
                frontier.push(neighbor_node, priority=heuristic(neighbor_node))

    return None, number_explored

def a_star_search(maze, start=(0, 0), goal=None):
    if goal is None:
        goal = (len(maze) - 1, len(maze[0]) - 1)
    
    heuristic = lambda node: node.path_cost + abs(goal[0] - node.state[0]) + abs(goal[1] - node.state[1])
    frontier = PriorityQueue()
    frontier.push(Node(start), priority=heuristic(Node(start)))
    explored = set()

    number_explored = 0
    while frontier.length() > 0:
        current_node = frontier.pop()
        current_state = current_node.state

        number_explored += 1
        if current_state == goal:
            return current_node, number_explored

        explored.add(current_state)
        for move in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            neighbor = (current_state[0] + move[0], current_state[1] + move[1])
            if valid_space(maze, neighbor) and neighbor not in explored:
                neighbor_node = Node(neighbor, parent=current_node)
                frontier.push(neighbor_node, priority=heuristic(neighbor_node))

    return None, number_explored

def display_results(maze, final_node, number_explored):
    if final_node is None:
        print("No path exists!\n")
        print_maze(maze)
    else:
        maze = copy.deepcopy(maze)
        node = final_node
        steps = 0
        while node.parent is not None:
            state = node.state
            maze[state[0]][state[1]] = 'X'
            steps += 1
            node = node.parent

        state = node.state
        maze[state[0]][state[1]] = 'X'
        print_maze(maze)

        print(f"Total steps on path: {steps}")
        print(f"Total states explored: {number_explored}\n")

# Main code
height, width = 10, 20
random.seed(0)
maze = mazegen(height, width)
print("Generated Maze:")
print_maze(maze)

print("\nGreedy Search:")
final_node, number_explored = greedy_search(maze)
display_results(maze, final_node, number_explored)

print("A* Search:")
final_node, number_explored = a_star_search(maze)
display_results(maze, final_node, number_explored)


Generated Maze:
..........#.....####
..#.....##.#.#....#.
..#..#...##....#....
#.#..............###
##....###.#...##....
.#......#.....#.....
...#..#...##........
...#...##..#..#.....
......#........#...#
............#.......

Greedy Search:
XXXXXXXX..#.....####
..#....X##.#.#....#.
..#..#.XX##....#....
#.#.....XXXXXXXXX###
##....###.#...##XXXX
.#......#.....#....X
...#..#...##.......X
...#...##..#..#...XX
......#........#..X#
............#.....XX
Total steps on path: 30
Total states explored: 33

A* Search:
XXXX......#.....####
..#X....##.#.#....#.
..#X.#...##....#....
#.#X.............###
##.X..###.#...##....
.#.XXXXX#.....#.....
...#..#XXX##........
...#...##X.#..#.....
......#..XXXXX.#...#
............#XXXXXXX
Total steps on path: 28
Total states explored: 115



In [18]:
import

class PartialEightQueensState:
    def __init(self, n=8):
        self.n = n

        self.possible_values [[i for i in range(0, self.n)] for _ in range(0, self.n)]
        self.final_values = [-1] * self.n 

    def is_goal(self):
        return all(value != -1 for value in self.final_values)

    def is_invalid(self):
        return any(len(values) == 0 for values in self.possible_values)

    def get_possible_values(self, column):
        return self.possible_values[column].copy()
    
    def get_final_state(self):
        if self.is_goal():
            return self.final_values
        else: 
            return -1
    
    def get_singleton_columns(self):

        return [index for index, values in enumerate(self.possible_values)
                if len(values) == 1 and self.final_values[index] == -1]
    
    def set_value(self, column, row):

        if row not in self.possible_values[column]:
            raise ValueError(f"{row} is not a valid choice for column {column}")
        
        state = copy.deepcopy(self)

        state.possible_values[column] = [row]
        state.final_values[column] = row

        state.possible_values[column] = [row]
        state.final_values[column] = row

        for update_col in range(0, column):

            if row in state.possible_values[update_col]:
                state.possible_values[update_col].remove(row)
            
            upper_diag = row + (column - update_col)
            if upper_diag in state.possible_values[update_col]
                state.possible_values[update_col].remove(upper_diag)
            
            lower_diag = row - (column - update_col)
            if lower_diag in state.possible_values[update_col]:
                state.possible_values[update_col].remove(lower_diag)
            
        
        for update_col in range(column + 1, state.n):

            if row in state.possible_values[update_col]:
                state.possible_values[update_col].remove(row)
            
            upper_diag = row + (update_col - column)
            if upper_diag in state.possible_values[update_col]:
                state.possible_values[update_col].remove(upper_diag)
            
            lower_diag = row - (update_col - column)
            if lower_diag in state.possible_values[update_col]:
                state.possible_values[update_col].remove(lower_diag)
            
        
        singleton_columns = state.get_singleton_columns()
        while len(singleton_columns) > 0:
            col = singleton_columns[0]
            state = state.set_value(col, state.possible_values[col][0])
            singleton_columns = state.get_singleton_columns()
        
        return state

SyntaxError: invalid syntax (1356450433.py, line 1)

In [ ]:
!pip install matplotlib.pyplot 

ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for matplotlib.pyplot


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
pip install --force-reinstall matplotlib numpy


  Using cached https://files.pythonhosted.org/packages/69/65/0d47953afa0ad569d12de5f65d964321c208492064c38fe3b0b9744f8d44/numpy-1.24.4-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/8b/5ba542fa83c90e09eac972fc9baca7a88e7e7ca4b221a89251954019308b/zipp-3.20.2-py3-none-any.whl
  Found existing installation: zipp 3.20.2
    Uninstalling zipp-3.20.2:
      Successfully uninstalled zipp-3.20.2
  Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateutil-2.9.0.post0
  Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Found

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [13]:
import numpy as np
np.array([5, 7, 4, 0, 2, 6, 1, 3], dtype=int)

array([5, 7, 4, 0, 2, 6, 1, 3])

In [ ]:
class EightQueensState:
    """This class represents a board in the eight queens puzzle"""
    def __init__(self, state=None, n=8):
        """
        :param state: pass in a numpy array of integers to set the state, otherwise will be generated randomly
        :param n: only used if state is not provided, determines size of board (default: 8)
        """
        if state is None:
            self.n = n
            state = np.random.randint(0, n, n)
        else:
            self.n = len(state)
        self.state = state

    @staticmethod
    def copy_replace(state, i, x):
        """This creates a copy of the state (important as numpy arrays are mutable) with column i set to x"""
        new_state = state.copy()
        new_state[i] = x
        return new_state

    @staticmethod
    def range_missing(start, stop, missing):
        """
        This creates a list of numbers with a single value missing
        e.g. range_missing(0, 8, 2) -> [0, 1, 3, 4, 5, 6, 7]
        """
        return list(range(start, missing)) + list(range(missing + 1, stop))

    def cost(self):
        """Calculates the number of pairs attacking"""
        count = 0
        for i in range(len(self.state) - 1):
            # for each queen, look in columns to the right
            # add one to the count if there is another queen in the same row
            count += np.any(self.state[i + 1:] == self.state[i])

            # add one to the count for each queen on the upper or lower diagonal
            upper_diagonal = self.state[i] + np.arange(1, self.n - i)
            lower_diagonal = self.state[i] - np.arange(1, self.n - i)
            count += np.any(self.state[i + 1:] == upper_diagonal)
            count += np.any(self.state[i + 1:] == lower_diagonal)
        return count

    def neighbourhood(self):
        """This generates every state possible by changing a single queen position"""
        neighbourhood = []
        for column in range(self.n):
            for new_position in self.range_missing(0, self.n, self.state[column]):
                new_state = self.copy_replace(self.state, column, new_position)
                neighbourhood.append(EightQueensState(new_state))

        return neighbourhood

    def random_neighbour(self):
        """Generates a single random neighbour state, useful for some algorithms"""
        column = np.random.choice(range(self.n))
        new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
        new_state = self.copy_replace(self.state, column, new_position)
        return EightQueensState(new_state)

    def is_goal(self):
        return self.cost() == 0

    def __str__(self):
        if self.is_goal():
            return f"Goal state! {self.state}"
        else:
            return f"{self.state} cost {self.cost()}"
state = EightQueensState()
print(state)
print(state.is_goal())
class HillClimber:
    """Applies the hill climbing algorithm to solve the 8 queens puzzle"""
    def __init__(self, state=EightQueensState()):
        self.state = state
        self.n = state.n

    @staticmethod
    def best_neighbour(state):
        """Gets all neighbours from the state, then returns the one with lowest cost"""
        return min(state.neighbourhood(), key=lambda x: x.cost())

    def hill_climb(self):
        """
        Repeatedly take the best neighbour of each state until we find a solution (or get stuck)
        :returns a goal state OR a local minimum (if all options are worse than the current one but it is not a goal)
        """
        if self.state.is_goal():
            return self.state

        while True:
            neighbour = HillClimber.best_neighbour(self.state)
            if neighbour.is_goal():
                return neighbour
            if neighbour.cost() < self.state.cost():
                self.state = neighbour
            else:
                # notice the method gives up if there are no better options
                return self.state

state = EightQueensState()
climber = HillClimber(state)
solution = climber.hill_climb()
print(solution)


class HillClimber:
    """Applies the hill climbing algorithm to solve the 8 queens puzzle"""
    def __init__(self, state=EightQueensState()):
        self.state = state
        self.n = state.n

    @staticmethod
    def best_neighbour(state):
        """Gets all neighbours from the state, then returns the one with lowest cost"""
        return min(state.neighbourhood(), key=lambda x: x.cost())

    def hill_climb(self):
        """
        Repeatedly take the best neighbour of each state until we find a solution (or get stuck)
        :returns a goal state OR a local minimum (if all options are worse than the current one but it is not a goal)
        """
        if self.state.is_goal():
            return self.state

        while True:
            neighbour = HillClimber.best_neighbour(self.state)
            if neighbour.is_goal():
                return neighbour
            if neighbour.cost() < self.state.cost():
                self.state = neighbour
            else:
                # notice the method gives up if there are no better options
                return self.state

    def iterative_hill_climb(self):
        """Your code goes here!"""
        pass
    
state = EightQueensState()
climber = HillClimber(state)
solution = climber.iterative_hill_climb()
print(solution)

[4 1 2 5 7 4 3 0] cost 4
False
Goal state! [5 2 6 3 0 7 1 4]


In [ ]:
import as np 

def is_valid
